In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter

In [2]:
df = pd.read_csv('train100k.csv')
df_test = pd.read_csv('test24k.csv')
items = pd.read_csv('items_id.csv')
items_date = pd.read_csv('items_date.csv')

In [3]:
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [4]:
def get_purchase_weigth(itemid, date, df_item=None):
    purchase_weigth = 0
#     df_item = df[df.itemids.str.contains(itemid)]
    for _, i in df_item.iterrows():
        purchase_weigth += 1 / (abs((i.date - date) / np.timedelta64(1, 'M')) + 1)
    return purchase_weigth

In [8]:
def analyze_user(userid, date):
    m = date.month
    if m < 10:
        m = f'0{m}'
    else:
        m = str(m)
    prods = pd.DataFrame({'itemid':[]})
    last_date = {}
    purchase_size = []
    for _, i in df[df.userid == userid].iterrows():
        one_purchase = []
        for id in i['itemids'].split():
            if id in last_date and last_date[id] < i.date:
                last_date[id] = i.date
            elif id not in last_date:
                last_date[id] = i.date
            if id not in one_purchase:
                one_purchase.append(id)
            prods = prods.append({'itemid':id}, ignore_index=True)
        purchase_size.append(len(one_purchase))
    counter = prods.value_counts()
    uniq_prods = prods.itemid.unique()
    prod_freq = pd.DataFrame({'itemid':[], 'weigth':[]})
    for id in uniq_prods:
        if items_date[items_date.itemid == int(id)][f'2020-{m}-15'].isna().all():
            w = 0
        else:
            w = items_date[items_date.itemid == int(id)][f'2020-{m}-15'].values[0]
        w = w * counter.at[id].values[0] / len(prods) / (abs((last_date[id] - date) / np.timedelta64(1, 'M')) + 1)
        
        prod_freq = prod_freq.append({'itemid':id, 'weigth':w}, ignore_index=True)
    
    prod_freq = prod_freq.sort_values(by='weigth', ascending=False) 
    purchase_size = int(np.round(np.array(purchase_size).mean()))
    return prod_freq[:purchase_size]

In [9]:
save_to = 'test_7try.csv'

for j, i in df_test.iterrows():
    if j % 10 == 0:
        print (j)
    if j % 25 == 0:
        df_test.to_csv(save_to, index=False)
    result = ' '
    prediction = analyze_user(i.userid, datetime.strptime(i.date, '%Y-%m-%d'))
    
    df_test.loc[df_test.userid == i.userid,'itemids'] = result.join(prediction.itemid)
    
df_test.to_csv(save_to, index=False)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

15510
15520
15530
15540
15550
15560
15570
15580
15590
15600
15610
15620
15630
15640
15650
15660
15670
15680
15690
15700
15710
15720
15730
15740
15750
15760
15770
15780
15790
15800
15810
15820
15830
15840
15850
15860
15870
15880
15890
15900
15910
15920
15930
15940
15950
15960
15970
15980
15990
16000
16010
16020
16030
16040
16050
16060
16070
16080
16090
16100
16110
16120
16130
16140
16150
16160
16170
16180
16190
16200
16210
16220
16230
16240
16250
16260
16270
16280
16290
16300
16310
16320
16330
16340
16350
16360
16370
16380
16390
16400
16410
16420
16430
16440
16450
16460
16470
16480
16490
16500
16510
16520
16530
16540
16550
16560
16570
16580
16590
16600
16610
16620
16630
16640
16650
16660
16670
16680
16690
16700
16710
16720
16730
16740
16750
16760
16770
16780
16790
16800
16810
16820
16830
16840
16850
16860
16870
16880
16890
16900
16910
16920
16930
16940
16950
16960
16970
16980
16990
17000
17010
17020
17030
17040
17050
17060
17070
17080
17090
17100
17110
17120
17130
17140
17150
17160
1717

In [10]:
df_test

,userid,date,itemids
0,6568788,2020-01-15,31315 59117 30834
1,2771293,2020-01-30,52979 21264 28998 44079
2,6761196,2020-01-29,52885 20113 58871 34454
3,3711380,2020-01-23,10596 20632 52438 15488 57937 55615 16059 4764...
4,9576135,2020-01-16,52663 32306 32142 13751 40875 16017
...,...,...,...
23995,6512522,2020-12-11,41964 54684 15662 52979 59750
23996,3846809,2020-12-17,21645 24170 43437 40875 12996 17753 57806
23997,6511355,2020-12-17,43437 31758 53136
23998,4591012,2020-12-15,43437 40875 44079 46345 21098


In [10]:
df_test

,userid,date,itemids
0,6568788,2020-01-15,31315 59117
1,2771293,2020-01-30,52979 21264 28998
2,6761196,2020-01-29,52885 20113 58871 34454
3,3711380,2020-01-23,10596 20632 52438 15488 57937 55615 16059 4764...
4,9576135,2020-01-16,52663 32306 32142 13751 40875 16017
...,...,...,...
23995,6512522,2020-12-11,41964 54684 15662 52979
23996,3846809,2020-12-17,21645 24170 43437 40875 12996 17753 57806
23997,6511355,2020-12-17,43437 31758
23998,4591012,2020-12-15,43437 40875 44079 46345 21098


In [ ]:
ids = items_date.itemid.unique()
dates = []
for i in range(1,13):
    if i < 10:
        dates.append(f'2020-0{i}-15')
    else:
        dates.append(f'2020-{i}-15')
for id in ids:
    df_item = df[df.itemids.str.contains(str(id))]
    cnt += 1
    for date in dates:
        if items_date[items_date.itemid == id][date].isna().any() == False:
            continue
        items_date.loc[items_date.itemid == id, date] = get_purchase_weigth(str(id), datetime.strptime(date, '%Y-%m-%d'), df_item)
    items_date.to_csv('items_date.csv', index=False)